# <center>__MÉTODOS NUMÉRICOS__</center>
## <center>__PROJETO DA UNIDADE 2__</center>

#### <center>__ALUNO:ISAAC ANTONIO SILVA BARBOSA__</center>

<div class="alert alert-block alert-info">
1. INTRODUÇÃO
</div>

A predição de dados tem como objetivo compreender o comportamento dos dados existente e gerar regras de predição para outras entradas de dados, e no nosso contexto iremos aplicar a regressão linear para predizer os dados do problema em questão. A regressão linear é uma função de primeiro grau, que busca entender o padrão de um conjunto de dados que possa ser descrito por essa função com uma variável. E para esse projeto será usado um conjunto de dados de pacientes com diabetes que possuem 442 amostra e 10 variáveis.

Durante o projeto iremos utilizar técnicas como: Regressão polinomial, caso especial de regressão linear, onde criamos recursos polinomiais antes da criação da regressão. Numba, projetado para ser utilizado como matrizes e funções NumPy, gera códigos especializados para distintos tipos de dados de array. Regularização, ajuda a resolver o problema de ajuste no machine learning. Entre outros métodos que acompanharemos durante a implementação.

<div class="alert alert-block alert-info">
2. DESCRIÇÃO DO PROBLEMA
</div>

Vários estudos foram desenvolvidos na área de aprendizado de máquina aplicado à saúde. Esse crescimento se da devido ao aumento da demanda por formas que possam facilitar os diagnósticos do paciente e diminuir o tempo dos médicos em suas prescrições.

E com o avanço constante da tecnologia é cada dia mais real a possibilidade desses diagnósticos serem feitos de forma computacional baseado em outros dados e utilizando tecnicas de aprendizado de máquina.

No Brasil foram desenvolvidos modelos preditivos de diabetes não diagnosticada a partir de 12.447 adultos, onde utilizaram cinco algoritimos de aprendizado de máquina. Como resultado eles obteram 11% de diabetes não diagnosticada. No conjunto de dados de teste 403 pessoas tinham diabetes não diagnoticada e 274 foram apontados como casos positivos.

<div class="alert alert-block alert-info">
3. MÉTODOS APLICADOS À SOLUÇÃO
</div>

Como principal método temos a Regressão Linear, que basicamente gera uma equação que descreve a relação entre uma ou mais variávies preditoras e a variável de resposta. Esse método encontra a reta que melhor representa o conjunto de entrada com a variável resposta. Podemos aplicar a regressão linear em diferentes contextos, como na análise de evasão escolar, variação da frequência cardíaca, valor de vendas de mercados, entre outras várias aplicações.

Também fazemos uso da regressão Polinomial partindo da ideia que podemos aproximar qualquer função com um polinômio. A ideia é bastante simples, iremos utilizar as variávis existentes para construir novas variáveis polinomiais e a regressão terá uma melhor aproximação de acordo com o grau do polinômio.

Para otimizar nosso código utilizamos a vetorização e o numba, pois ao usar uma implementação vetorizada podemos tornar o processo muito mais rápido em comparação com a nossa implementação convencional e para isso utilizamos a biblioteca Numpy. Já o numba é um compilador JIT que converte um subconjunto de Python e NumPy em um código de máquina mais rápido usando LLVM.

E a regularização ajuda a resolver o problema de ajuste no aprendizado de máquina. O modelo complexo não é tão bom desempenho nos dados de teste, já o modelo simples é a generalização muito pobre dos dados. É necessário optar por um dos modelos e a regularização ajuda a escolher a complexidade do modelo preferido afim que ele seja melhor nas previsões.

<div class="alert alert-block alert-info">
4. IMPLEMENTAÇÃO
</div>

In [1]:
from sklearn import datasets, linear_model, metrics 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math, scipy, numpy as np
from scipy import linalg

In [49]:
#dados
data = datasets.load_diabetes() 

In [50]:
feature_names=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [51]:
trn,test,y_trn,y_test = train_test_split(data.data, data.target, test_size=0.2)

In [52]:
trn.shape, test.shape

((353, 10), (89, 10))

<div class="alert alert-block alert-info">
Regressão Linear com Scikit Learn
</div>

In [6]:
regr = linear_model.LinearRegression()
%timeit regr.fit(trn, y_trn)

529 µs ± 35.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
pred = regr.predict(test)

In [53]:
def regr_metrics(act, pred):
    return (math.sqrt(metrics.mean_squared_error(act, pred)), 
     metrics.mean_absolute_error(act, pred))

In [9]:
regr_metrics(y_test, regr.predict(test))

(56.89722162606827, 47.63052746843751)

In [10]:
trn.shape

(353, 10)

In [11]:
poly = PolynomialFeatures(include_bias=False)

In [54]:
trn_feat = poly.fit_transform(trn)

In [55]:
', '.join(poly.get_feature_names(feature_names))

'age, sex, bmi, bp, s1, s2, s3, s4, s5, s6, age^2, age sex, age bmi, age bp, age s1, age s2, age s3, age s4, age s5, age s6, sex^2, sex bmi, sex bp, sex s1, sex s2, sex s3, sex s4, sex s5, sex s6, bmi^2, bmi bp, bmi s1, bmi s2, bmi s3, bmi s4, bmi s5, bmi s6, bp^2, bp s1, bp s2, bp s3, bp s4, bp s5, bp s6, s1^2, s1 s2, s1 s3, s1 s4, s1 s5, s1 s6, s2^2, s2 s3, s2 s4, s2 s5, s2 s6, s3^2, s3 s4, s3 s5, s3 s6, s4^2, s4 s5, s4 s6, s5^2, s5 s6, s6^2'

In [14]:
trn_feat.shape

(353, 65)

In [56]:
regr.fit(trn_feat, y_trn)

LinearRegression()

In [57]:
regr_metrics(y_test, regr.predict(poly.fit_transform(test)))

(59.382320009226454, 46.31452979755305)

In [17]:
%timeit poly.fit_transform(trn)

268 µs ± 24.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
%matplotlib inline

In [61]:
import math, numpy as np, matplotlib.pyplot as plt
from scipy import ndimage
from numba import jit, vectorize, guvectorize, cuda, float32, void, float64

In [62]:
def proc_python(xx,yy):
    zz = np.zeros(nobs, dtype='float32')
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [63]:
nobs = 10000
x = np.random.randn(nobs).astype('float32')
y = np.random.randn(nobs).astype('float32')

In [64]:
%timeit proc_python(x,y)

62.4 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
def proc_numpy(x,y):
    z = np.zeros(nobs, dtype='float32')
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    z = z * ( z - .88 ) 
    return z

In [66]:
#retorna true se duas matrizes sao iguais em termos de elemento dentro de uma tolerancia.
np.allclose( proc_numpy(x,y), proc_python(x,y), atol=1e-4 )

True

In [67]:
%timeit proc_numpy(x,y)

31.3 µs ± 551 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [68]:
@jit()
def proc_numba(xx,yy,zz):
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [69]:
z = np.zeros(nobs).astype('float32')
np.allclose( proc_numpy(x,y), proc_numba(x,y,z), atol=1e-4 )

True

In [70]:
%timeit proc_numba(x,y,z)

25.3 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [71]:
@vectorize
def vec_numba(x,y):
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    return z * ( z - .88 )

In [72]:
np.allclose(vec_numba(x,y), proc_numba(x,y,z), atol=1e-4 )

True

In [73]:
%timeit vec_numba(x,y)

5.44 µs ± 386 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [74]:
@jit(nopython=True)
def vec_poly(x, res):
    m,n=x.shape
    feat_idx=0
    for i in range(n):
        v1=x[:,i]
        for k in range(m): res[k,feat_idx] = v1[k]
        feat_idx+=1
        for j in range(i,n):
            for k in range(m): res[k,feat_idx] = v1[k]*x[k,j]
            feat_idx+=1

In [77]:
trn = np.asfortranarray(trn)
test = np.asfortranarray(test)

In [34]:
m,n=trn.shape
n_feat = n*(n+1)//2 + n
trn_feat = np.zeros((m,n_feat), order='F')#features de treino
test_feat = np.zeros((len(y_test), n_feat), order='F')#features de teste

In [35]:
vec_poly(trn, trn_feat)
vec_poly(test, test_feat)

In [36]:
regr.fit(trn_feat, y_trn)

LinearRegression()

In [37]:
regr_metrics(y_test, regr.predict(test_feat))

(61.567417086616004, 49.476799912692506)

In [38]:
%timeit vec_poly(trn, trn_feat)

6.15 µs ± 438 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [39]:
%timeit poly.fit_transform(trn) #ajustar os dados e transforma-los

272 µs ± 4.76 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



<div class="alert alert-block alert-info">
Regularização e ruído
</div>

In [40]:
reg_regr = linear_model.LassoCV(n_alphas=10) #modelo linear de laco com ajuste iterativo ao longo de um caminho de regularizacao.

In [41]:
reg_regr.fit(trn_feat, y_trn)

C:\Users\isaac\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1224.1877532505896, tolerance: 167.0623319148936
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\isaac\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 59544.44129698357, tolerance: 161.81857243816253
  model = cd_fast.enet_coordinate_descent_gram(


LassoCV(n_alphas=10)

In [42]:
reg_regr.alpha_ 

0.01016995679378688

In [43]:
regr_metrics(y_test, reg_regr.predict(test_feat))

(56.4337123254913, 46.9115793796714)

##### **Ruído**

Agora vamos adicionar algum ruído aos dados

In [44]:
idxs = np.random.randint(0, len(trn), 10) #retornando numeros aleatorios

In [45]:
y_trn2 = np.copy(y_trn)
y_trn2[idxs] *= 10 # label noise

In [46]:
regr = linear_model.LinearRegression()
regr.fit(trn, y_trn)
regr_metrics(y_test, regr.predict(test))

(56.89722162606828, 47.630527468437506)

In [47]:
regr.fit(trn, y_trn2)
regr_metrics(y_test, regr.predict(test))

(69.08205640773274, 56.97976191213168)

In [48]:
hregr = linear_model.HuberRegressor()
hregr.fit(trn, y_trn2)
regr_metrics(y_test, hregr.predict(test))

C:\Users\isaac\anaconda3\lib\site-packages\sklearn\linear_model\_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


(57.305643660423655, 48.03492887432899)